In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import csv
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer 
from collections import Counter
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

0     7072
1    27273
2    79582
3    32927
4     9206
dtype: int64

In [2]:
data = pd.read_csv("data.tsv", sep='\t')
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'

In [6]:
data['Sentiment'].unique()
sent_labels=data['Sentiment'].unique()
sent_labels

array([1, 2, 3, 4, 0], dtype=int64)

In [7]:
numoflabels=Counter(data['Sentiment'])
arroul=((sorted(list(numoflabels.items()))))
arroul

[(0, 7072), (1, 27273), (2, 79582), (3, 32927), (4, 9206)]

In [3]:
x = data.iloc[:, :-1]
y = data.iloc[:,-1:]
x

,PhraseId,SentenceId,Phrase
0,1,1,A series of escapades demonstrating the adage ...
1,2,1,A series of escapades demonstrating the adage ...
2,3,1,A series
3,4,1,A
4,5,1,series
...,...,...,...
156055,156056,8544,Hearst 's
156056,156057,8544,forced avuncular chortles
156057,156058,8544,avuncular chortles
156058,156059,8544,avuncular


In [9]:
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [4]:
#REMOVE STOPWORDS
sw=set(stopwords.words('english'))
#newdata=x.copy()
#newdata['Phrase'] = newdata.Phrase.astype(str).str.replace("[^\w\s]", "").str.lower()
#newdata['Phrase'] = newdata['Phrase'].apply(lambda x: ' '.join([item for item in x.split() if item not in sw]))
df=x.copy()
df['Phrase'] = df.Phrase.astype(str).str.replace("[^\w\s]", "").str.lower()
df['Phrase'] = df.Phrase.astype(str).str.replace('\d+', '')
df['Phrase'] = df['Phrase'].apply(lambda x: [item for item in x.split() if item not in sw])
#docs=newdata['Phrase'].tolist()


In [11]:
df

,PhraseId,SentenceId,Phrase
0,1,1,"[series, escapades, demonstrating, adage, good..."
1,2,1,"[series, escapades, demonstrating, adage, good..."
2,3,1,[series]
3,4,1,[]
4,5,1,[series]
...,...,...,...
156055,156056,8544,[hearst]
156056,156057,8544,"[forced, avuncular, chortles]"
156057,156058,8544,"[avuncular, chortles]"
156058,156059,8544,[avuncular]


In [5]:
#STEMMING
stemm = SnowballStemmer("english")
wstokenizer = WhitespaceTokenizer()
def stemming(text):
    return [stemm.stem(w) for w in wstokenizer.tokenize(str(text))]
df['Phrase'] = df['Phrase'].apply(lambda x: [stemm.stem(y) for y in x]) 
df

,PhraseId,SentenceId,Phrase
0,1,1,"[seri, escapad, demonstr, adag, good, goos, al..."
1,2,1,"[seri, escapad, demonstr, adag, good, goos]"
2,3,1,[seri]
3,4,1,[]
4,5,1,[seri]
...,...,...,...
156055,156056,8544,[hearst]
156056,156057,8544,"[forc, avuncular, chortl]"
156057,156058,8544,"[avuncular, chortl]"
156058,156059,8544,[avuncular]


In [6]:
df['Phrase'] = [' '.join(i) if isinstance(i, list) else i for i in df['Phrase']]
df

,PhraseId,SentenceId,Phrase
0,1,1,seri escapad demonstr adag good goos also good...
1,2,1,seri escapad demonstr adag good goos
2,3,1,seri
3,4,1,
4,5,1,seri
...,...,...,...
156055,156056,8544,hearst
156056,156057,8544,forc avuncular chortl
156057,156058,8544,avuncular chortl
156058,156059,8544,avuncular


In [7]:
#SPLITTING INTO TRAIN AND TEST OUR MAIN DATA
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Phrase'], 
                                                    y, 
                                                    random_state=0)

In [21]:
#just for another case dont need to run
#COUNTV WITH NORM DATA
#I USE IN THIS CASE 5000 CAUSE OF MEMORY PROBLEM AND CAUSE I USE DATAFRAME FORMAT EXCEPT SPARSE MATRIX
cv = CountVectorizer(analyzer='word').fit(df['Phrase'][:50000])
cvtrain = cv.transform(df['Phrase'][:50000])
cvtrain.shape
cvtrain=pd.DataFrame(cvtrain.toarray(), columns=cv.get_feature_names())
cvtrain


,aaliyah,abandon,abel,abhorr,abil,abl,abornin,abort,abound,abridg,...,zelda,zemecki,zero,zhang,zhuangzhuang,zing,zinger,zingerfil,zip,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#CV WITH TRAIN DATA
cv = CountVectorizer().fit(X_train)
cvtrain_vect = cv.transform(X_train)
cvtrain_vect.shape


(117045, 11406)

In [34]:
cv = CountVectorizer(binary=True)
cv.fit(X_train)
X = cv.transform(X_train)
X_test = cv.transform(X_test)

In [36]:
final_model = LogisticRegression(C=0.05)
final_model.fit(X, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_train, final_model.predict(X_test)))

C:\Users\User\Anaconda3\envs\ML\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


ValueError: Found input variables with inconsistent numbers of samples: [117045, 39015]

In [33]:
#LOGISTIC REGRESSION CV TRAIN DATA (for sparse matrix format)
cvtest_vect = cv.transform(X_test)
lrcv = LogisticRegression(C=0.05)
lrcv.fit(X_train, y_train)
acc_lr_cv = accuracy_score(y_test, lrcv.predict(X_test))
print('Accuracy: %.2f%%' % (acc_lr_cv * 100))

ValueError: could not convert string to float: 'illus'

In [14]:
#MULTINOMIALNB CV TRAIN DATA
mnb = MultinomialNB(alpha=0.1)
mnb.fit(cvtrain_vect, y_train)
y_pred_cv = mnb.predict(cv.transform(X_test))
acc_mnb_cv=accuracy_score(y_test, y_pred_cv)
print('Accuracy: %.2f%%' % (acc_mnb_cv * 100))

Accuracy: 60.83%


In [15]:
#SVM TO CV TRAIN DATA
svmcv = svm.SVC()
svmcv.fit(cvtrain_vect, y_train)
svm_pred_cv = svmcv.predict(cvtest_vect)
acc_svm_cv=accuracy_score(cvtest_vect, svm_pred_cv)
print('Accuracy: %.2f%%' % (acc_svm_cv * 100))

In [22]:
#TFVECT WITH NORM DATA
tfv = TfidfVectorizer(min_df=10)
tftrain = tfv.fit_transform(df['Phrase'][:50000])
tftrain= pd.DataFrame(tftrain.toarray(), columns=tfv.get_feature_names())
tftrain


,aaliyah,abandon,abil,abl,abornin,abort,absolut,absorb,absurd,abus,...,young,younger,youth,zani,zeal,zelda,zemecki,zhang,zinger,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#TFVECT WITH TRAIN  DATA
tfv = TfidfVectorizer(min_df=3).fit(X_train)
tftrain_vect = tfv.transform(X_train)

In [16]:
#LOGISTIC REGRESSION TF TRAIN DATA
tftest_vect = tfv.transform(X_test)
lrtf = LogisticRegression(C=0.05)
lrtf.fit(tftrain_vect, y_train)
acc_lr_tf = lrtf.score(y_test, lrtf.predict(tftest_vect))
print('Accuracy: %.2f%%' % (acc_lr_tf * 100))

C:\Users\User\Anaconda3\envs\ML\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


ValueError: X has 1 features per sample; expecting 11127

In [37]:
#SVM TF TRAIN DATA
svmtf = svm.SVC()
svmtf.fit(tftrain_vect, y_train)
svm_pred_tf = svmtf.predict(tftest_vect)
acc_svm_tf=accuracy_score(tftest_vect, svm_pred_tf)
print('Accuracy: %.2f%%' % (acc_svm_tf * 100))

C:\Users\User\Anaconda3\envs\ML\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


TypeError: len() of unsized object

In [17]:
#MULTINOMIALNB TF TRAIN DATA
mnbTF = MultinomialNB(alpha=0.1)
mnbTF.fit(tftrain_vect, y_train)
y_pred_tf = mnbTF.predict(tfv.transform(X_test))
acc_mnb_tf=(accuracy_score(y_test, y_pred_tf))
print('Accuracy: %.2f%%' % (acc_mnb_tf * 100))

C:\Users\User\Anaconda3\envs\ML\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy: 59.65%


In [24]:
#SPLIT INTO TRAIN TEST TF  DATA (MAIN DATA TF VECT)
from sklearn.model_selection import train_test_split
tfx_train, tfx_test, tfy_train, tfy_test = train_test_split(tftrain, y[:50000], test_size=0.3)
tfx_train

,aaliyah,abandon,abil,abl,abornin,abort,absolut,absorb,absurd,abus,...,young,younger,youth,zani,zeal,zelda,zemecki,zhang,zinger,zone
33746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
#GAUSIAN NB FOR TF DATA
#GAUSIANNB DONT WORK WITH SPARSE MATRIX AND I VECTORIZED MY MAIN DATA WITH CV AND TF USE DATAFRAME AND THEN SPLIT THEM INTO TT THEN CHECK EVERY MODEL 
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(tfx_train, tfy_train)
nbpred = gnb.predict(tfx_train)
gnbaccuracy = accuracy_score(tfy_train, nbpred)
print(gnbaccuracy)

C:\Users\User\Anaconda3\envs\ML\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.20942857142857144


In [ ]:
#SVC FOR TF DATA
svmclf = svm.SVC(kernel='linear', C = 1.0)
svmclf.fit(tfx_train, tfy_train)
svmpred = svmclf.predict(tfx_test)
svmaccuracy=accuracy_score(tfy_test, svmpred)
print (svmaccuracy)

C:\Users\User\Anaconda3\envs\ML\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [ ]:
#LOGISTIC REG FOR TF DATA
from sklearn.linear_model import LogisticRegression
lregr = LogisticRegression()
lregr.fit(tfx_train, tfy_train)
lrpred = lregr.predict(tfy_test)
lraccuracy = lregr.score(tfx_test, tfy_test)
print(lraccuracy)

In [ ]:
#SPLIT INTO TRAIN TEST CV DATA

cx_train, cx_test, cy_train, cy_test = train_test_split(cvtrain, y[:5000], test_size=0.3)
cx_train

In [ ]:
#LOGISTIC REG FOR CV DATA
from sklearn.linear_model import LogisticRegression
CV_lregr = LogisticRegression()
CV_lregr.fit(cx_train, cy_train)
CV_lrpred = CV_lregr.predict(cx_test)
CV_lraccuracy = CV_lregr.score(cx_test, cy_test)
print(CV_lraccuracy)

In [ ]:
#SVC FOR CV DATA
from sklearn.metrics import accuracy_score
from sklearn import svm
CV_svmclf = svm.SVC(kernel='linear', C = 1.0)
CV_svmclf.fit(cx_train, cy_train)
CV_svmpred = CV_svmclf.predict(cx_test)
CV_svmaccuracy=accuracy_score(cy_test, CV_svmpred)
print (CV_svmaccuracy)

In [ ]:
#GAUSIAN NB FOR CV DATA
from sklearn.naive_bayes import GaussianNB
CV_gnb = GaussianNB()
CV_gnb.fit(cx_train, cy_train)
CV_nbpred = CV_gnb.predict(cx_train)
CV_gnbaccuracy = accuracy_score(cy_train, CV_nbpred)
print(CV_gnbaccuracy)

In [ ]:
#CHECK 
print('TfidfVectorizer accuracy','\n',
      "Logistic regression accuracy: " ,lraccuracy,'\n',
      "SVM  accuracy: ",svmaccuracy,'\n',
      "GausianNB  accuracy: ",gnbaccuracy,'\n',
      "MultinomialNB  accuracy: ",acc_mnb_tf,'\n',
      "Logistic regression  accuracy 2(matrix): " ,acc_lr_tf,'\n',
      "SVM  accuracy(matrix): ",acc_svm_tf,'\n'
     )

print('CountVectorizer accuracy','\n',
      "Logistic regression accuracy: " ,CV_lraccuracy,'\n',
      "SVM  accuracy: ",CV_svmaccuracy,'\n',
      "GausianNB  accuracy: ",CV_gnbaccuracys,'\n',
      "MultinomialNB  accuracy: ",acc_mnb_cv,'\n',
      "Logistic regression  accuracy 2(matrix): " ,acc_lr_cv,'\n',
      "SVM  accuracy(matrix): ",acc_svm_cv,'\n'
     )

